In [1]:
import sys
import pickle 

sys.path.append(sys.path[0] + "/..")  # Adds higher directory to python modules path.

import numpy as np
from Scripts import renormalize, scomplex
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm

plt.rcParams["text.usetex"] = False


colors = ["#003F5C","#2F4B7C","#665191","#A05195","#D45087","#F95D6A","#FF7C43","#FFA600"]
colors_sequential = colors + colors + colors + colors 
colors = ["#02405c", "#fea600", "#a20655", "#5e96c5", "#4b3596", "#fa6ca9", "#8d6cf6"]
colors_curves = colors+ colors+ colors+ colors


### Functions

In [17]:
def hg_to_sc(H):
    H.cleanup()

    sc = {}
    sc["nodes"] = np.sort(np.array([H.nodes]).T,0)
    sc["n0"] = sc["nodes"].shape[0]
    keys = ["edges","faces","tetrahedra","4-simplices"]

    for k in keys:
        sc[k] = []
    for e in H.edges.members():
        if len(e) <= 5:
            sc[keys[len(e)-2]].append(list(e))

    for i,k in enumerate(keys):
        if len(sc[k]) == 0:
            sc[k] = np.zeros((0,i+2))
        else:
            sc[k] = np.unique(np.sort(np.array(sc[k]),1),axis =0)
        
        sc[f"n{i+1}"] = sc[k].shape[0] 
    return sc

In [7]:
def laplacians(dim = 2):
    return [str(a)+str(b) for a in range(dim+1) for b in range(dim+1) if a != b]

# XGI Hypergraphs test

In [14]:
names_hg = ['congress-bills', 'contact-high-school', 'contact-primary-school', 'diseasome', 'disgenenet','email-enron', 'email-eu', 'hospital-lyon', 'house-bills', 'house-committees', 'hypertext-conference', 'invs13', 'invs15', 'malawi-village','ndc-classes', 'ndc-substances', 'science-gallery', 'senate-bills', 'senate-committees','sfhh-conference']
true_names_hg = ['US Congress Bills', 'High school contacts', 'Primary school contact', 'Diseasome', 'Disgenenet', 'Enron email', 'Eu email', 'Hospital contacts', 'House bills', 'House committees', 'Hypertext contacts', 'InVS13', 'InVS15', 'Malawi contacts', 'Classes of NDC', 'Substances of NDC', 'Science Gallery', 'Senate bills', 'Senate committees', 'SFHH conference']

Import hypergraphs

In [16]:
with open("../Data/Datasets/xgi/xgi_sc.pkl", "rb") as f:
    scs = pickle.load(f)

<h1>Compute Laplacians an Sp Heats</h1>

In [17]:
N_data = len(names_hg)
print('Number of datasets =', N_data)

Number of datasets = 20


In [ ]:
taumin = -3 # Heat curve starts from 10**taumin
taumax = 5 # Heat curve ends at 10**taumax
ntau = 200 # Number of taus to consider in the interval
SP_HEATS = {}
for name in tqdm(names_hg):
    sc = scs[name]

    smax = 0 # Hypergraph dimension
    for i in range(5):
        if sc[f'n{i}'] != 0:
            smax +=1
    # Define strings which specify the cross-order Laplacians to consider
    laplacians_types = laplacians(smax - 1)

    C_curves = {l : [] for l in laplacians_types}
    for idl,l in tqdm(enumerate(laplacians_types)):    
        L = scomplex.XO_laplacian_hg(sc, k=int(l[0]), l=int(l[1]))    
        D,__ = np.linalg.eigh(L)
        D = np.abs(D)
        entropic_susceptibility,tau_space,__  = renormalize.compute_entropic_C(D,taumin,taumax,ntau)
        C_curves[l] = entropic_susceptibility
    SP_HEATS[name] = C_curves


Save specific heats for the hypergraphs

In [11]:
with open('../Data/C_curves/RealData/xgi/xgi_spheats.pkl', "wb") as f:
    pickle.dump(SP_HEATS, f)

In [ ]:
taumin = -3 # Heat curve starts from 10**taumin
taumax = 5 # Heat curve ends at 10**taumax
ntau = 200 # Number of taus to consider in the interval
nrep = 5

SP_HEATS_CM = {}

for name in tqdm(names_hg):
    sc = scs[name]
    smax = 0
    for i in range(5):
        if sc[f'n{i}'] != 0:
            smax +=1


    # Define strings which specify the cross-order Laplacians to consider
    laplacians_types = laplacians(smax - 1)

    C_curves = {l : [] for l in laplacians_types}

    for idl,l in tqdm(enumerate(laplacians_types)): 
        A = scomplex.adjacency_of_order_hg(sc, k=int(l[0]), l=int(l[1]))
        for n in tqdm(range(nrep)):
            # # Configuration model
            Gcm = nx.Graph(A)
            Gcm = nx.configuration_model([val for (__, val) in Gcm.degree()])
            L = nx.laplacian_matrix(Gcm).todense()
            D,__ = np.linalg.eigh(L)
            D = np.abs(D)
            entropic_susceptibility,tau_space,__  = renormalize.compute_entropic_C(D,taumin,taumax,ntau)
            C_curves[l].append(entropic_susceptibility)

    SP_HEATS_CM[name] = C_curves


In [ ]:
with open('../Data/C_curves/RealData/xgi/xgi_spheats_cm.pkl', "wb") as f:
    pickle.dump(SP_HEATS_CM, f)